In [17]:
import jax
import projects.charformer.data as data
from importlib import reload
data = reload(data)
import sys
import numpy
import jax
import jax.numpy as jnp
numpy.set_printoptions(threshold=sys.maxsize)
# %pip install tqdm


2024-09-01 22:08:17.028819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 22:08:17.087096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 22:08:17.104638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 22:08:18.838991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [19]:
ds = data.CharDataset(data.CharDataset.get_default_config())

In [20]:
# !  wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt

In [21]:
ds.create_packed_records(input_file_path='projects/charformer/TinyStoriesV2-GPT4-train.txt', output_dir='projects/charformer/data/tfrecords/', custom_delimiter='<|endoftext|>')

FileNotFoundError: [Errno 2] No such file or directory: 'projects/charformer/TinyStoriesV2-GPT4-train.txt'

In [22]:
retokenized = ds.load_and_retokenize_tfrecord(file_path='projects/charformer/data/tfrecords/record_40.tfrecord')

In [23]:
for r in retokenized:
    print(r)

One day, a little bug named Tim went for a walk. It was a cold day, and he saw a big tree. Tim thought the tree would be a good place to stay warm. Inside the tree, he found a small room with a cobweb in the corner.
Tim saw another bug in the room. She was a spider named Sue. Sue said, "Hi, Tim! Do you like my cobweb?" Tim said, "Yes, it is very nice." Sue told Tim that she had a big fruit in the other room. "But it is old and starting to rot," she said. "We should eat it soon."
The next day, Tim and Sue went to the other room to eat the fruit. They saw that it was very big and smelled bad. Tim said, "Oh no, the fruit is all rotten!" Sue said, "That's okay, we can still eat it." They ate the fruit and felt warm and happy. The cold day did not bother them anymore.
<|endoftext|
Once upon a time, there was a little bathtub. The bathtub lived in a dark room. The bathtub was sad because it was always alone.
One day, a little girl named Lucy came into the room. She saw the bathtub and said, 

In [24]:
iter = ds.create_iterator('projects/charformer/data/tfrecords/record_*.tfrecord', batch_size=8)

In [25]:
batch = next(iter)

In [103]:
import minformer.model as model
import functools
model = reload(model)

In [111]:
cfg = model.Config(
    d_model=1024,
    ffw_multiplier=4,
    query_heads=8,
    key_heads=8,
    num_layers=8,
    key_dim=16,
    vocab_size=256,
    causal=True,
    use_attn_kernel=True,
    rules=model.fsdp_rules,
    mesh=model.create_mesh()
)

In [112]:
weights = model.Weights.init(cfg, jax.random.PRNGKey(0), cfg.mesh, model.fsdp_rules)

In [113]:

import math
sum(jax.tree.leaves(jax.tree.map(lambda x: math.prod(jnp.shape(x)), weights))) / 1e9

0.035135488

In [114]:
# jax.tree.map(lambda w: print(w.shape, w.dtype, jnp.mean(w)), weights)

In [115]:
step = jax.jit(model.update_step, static_argnames='cfg')
step = functools.partial(step, cfg=cfg)

In [116]:
batch['segment_ids'].shape

(8, 4096)

In [117]:
loss, weights = step(weights, batch['x'], batch['segment_ids'], batch['y'])

In [118]:
# Does loss go down!
for i in range(0, 10):
    loss, weights = step(weights, batch['x'], batch['segment_ids'], batch['y'])
    print(loss)

8.0886965
7.8215375
7.5749617
7.3384504
7.116937
6.903502
6.6988564
6.5027857
6.3136754
6.139119


In [60]:
rules = {
    'd': 'x',
    'f': None}
pspec = jax.sharding.PartitionSpec('d', 'f')

x = jax.sharding.PartitionSpec(*(rules[l] for l in pspec))
x

PartitionSpec('x', None)

In [46]:

def f(x):
    y = x @ x
    y = jax.lax.with_sharding_constraint(y, shardings=jax.sharding.NamedSharding(mesh, ('x', 'y')))
    y += 1
    return y

x = jnp.ones((8, 8))

print(jax.jit(f).lower(x).as_text('hlo'))

HloModule jit_f, entry_computation_layout={(f32[8,8]{1,0})->f32[8,8]{1,0}}, frontend_attributes={xla.sdy.meshes={}}

ENTRY main.6 {
  Arg_0.1 = f32[8,8]{1,0} parameter(0)
  dot.4 = f32[8,8]{1,0} dot(Arg_0.1, Arg_0.1), lhs_contracting_dims={1}, rhs_contracting_dims={0}
  constant.2 = f32[] constant(1)
  broadcast.3 = f32[8,8]{1,0} broadcast(constant.2), dimensions={}
  ROOT add.5 = f32[8,8]{1,0} add(dot.4, broadcast.3)
}




In [134]:
print(jax.__version__)

0.4.32.dev20240831


In [4]:
from jax.experimental.pallas.ops.tpu import flash_attention

In [8]:
batch = 4
num_heads = 8
q_seq_len = 4096
kv_seq_len = 4096
key_size = 256
key1, key2, key3, key4, key5 = jax.random.split(jax.random.PRNGKey(0), 5)
q = jax.random.normal(key1, (batch, num_heads, q_seq_len, key_size), dtype=jnp.float32)
k = jax.random.normal(key2, (batch, num_heads, kv_seq_len, key_size), dtype=jnp.float32)
v = jax.random.normal(key3, (batch, num_heads, kv_seq_len, key_size), dtype=jnp.float32)
q_segment_ids = jax.random.randint(key4, (batch, q_seq_len), 0, 2)
kv_segment_ids = jax.random.randint(key5, (batch, kv_seq_len), 0, 2)
segment_ids = flash_attention.SegmentIds(q_segment_ids, kv_segment_ids)
flash_attention.flash_attention(q, k, v, segment_ids=segment_ids, causal=True)

Array([[[[ 4.37500000e-01, -4.51171875e-01,  1.90625000e+00, ...,
          -1.26562500e+00,  2.69531250e-01,  5.97656250e-01],
         [ 4.37500000e-01, -4.51171875e-01,  1.90625000e+00, ...,
          -1.26562500e+00,  2.69531250e-01,  5.97656250e-01],
         [ 4.37500000e-01, -4.51171875e-01,  1.90625000e+00, ...,
          -1.26562500e+00,  2.69531250e-01,  5.97656250e-01],
         ...,
         [-1.79270184e+00, -9.00238633e-01,  1.17744589e+00, ...,
          -1.25635004e+00, -1.52953541e+00,  2.41711688e+00],
         [ 5.43850839e-01,  6.42475843e-01,  1.33804083e-01, ...,
           1.36632419e+00,  3.25035095e-01,  8.06720734e-01],
         [-4.04746294e-01, -5.18119752e-01, -5.59119105e-01, ...,
           3.79983574e-01,  1.83797240e+00, -7.52215564e-01]],

        [[-8.94531250e-01, -1.49414062e-01,  6.54296875e-02, ...,
           1.29687500e+00,  3.96484375e-01, -4.51171875e-01],
         [-8.94531250e-01, -1.49414062e-01,  6.54296875e-02, ...,
           1.29687500e

In [1]:
from functools import partial

import jax
from jax.experimental import pallas as pl
import jax.numpy as jnp
import numpy as np

In [3]:
def add_vectors_kernel(x_ref, y_ref, o_ref):
  x, y = x_ref[...], y_ref[...]
  o_ref[...] = x + y
  
@jax.jit
def add_vectors(x: jax.Array, y: jax.Array) -> jax.Array:
  return pl.pallas_call(
      add_vectors_kernel,
      out_shape=jax.ShapeDtypeStruct(x.shape, x.dtype)
  )(x, y)
add_vectors(jnp.arange(8), jnp.arange(8))

Array([ 0,  2,  4,  6,  8, 10, 12, 14], dtype=int32)

In [10]:
segment_ids = jnp.array([[1,1,1,2,2,0,0], [1,1,2,2,2,0,0]])

In [15]:
(segment_ids[:, None, :] == segment_ids[:, :, None]).astype(jnp.int32)

Array([[[1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 1]],

       [[1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 1]]], dtype=int32)

In [14]:
jnp.tril(jnp.ones((7, 7)))

Array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

In [16]:
jax.experimental.shard_map

<module 'jax.experimental.shard_map' from '/home/sholto/.local/lib/python3.10/site-packages/jax/experimental/shard_map.py'>